<a href="https://colab.research.google.com/github/roshantac/ERA/blob/main/Session08/Cifar_using_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jun 23 05:51:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%matplotlib inline
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

In [4]:
!ls

base_functions.py  dataset.py  model.py     sample_data  utils.py
data		   main.py     __pycache__  training.py


Import DataSet
# New Section

In [5]:
from dataset import *
trainloader, testloader, classes = getData()

Files already downloaded and verified
Files already downloaded and verified


# Define your model Here

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from base_functions import *
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.C1 = Conv2d_BN(3,22,dropout=0.1, padding=1)     # 32    C1
        self.C2 = Conv2d_BN(22,32,dropout=0.1, padding=1)    # 32 #. C2
        self.c_3= Conv1x1(32,22)
        self.P1 = Maxpooling(2)
        self.C3 = Conv2d_BN(22,32,dropout=0.1, padding=1)  #16 #. C3
        self.C4 = Conv2d_BN(32,22,dropout=0.1, padding=1)   # 16 #  C4
        self.C5 = Conv2d_BN(22,42,dropout=0.1, padding=1)#16#. C5
        self.c_6= Conv1x1(42,32)                           #  C6
        self.P2 = Maxpooling(2)
        self.C7 = Conv2d_BN(32,22, dropout=0.1, padding=1) # 8.#. C7
        self.C8 = Conv2d_BN(22,32, dropout=0.1, padding=1) # 8  #  C8
        self.C9 = Conv2d_BN(32,22, padding=1) #. C9
        self.gap= gap(8)
        self.FC = Conv1x1(22,10)


    def forward(self,x):
      #32
      x = self.C1(x)
      x = self.C2(x)
      x = self.c_3(x)
      x = self.P1(x)
      #16
      s = self.C3(x)
      x = self.C4(s)
      x = self.C5(x)
      x = self.c_6(x) + s
      s2 = self.P2(x)
      #8
      s = self.C7(s2)
      x = self.C8(s) + s2
      x = self.C9(x) + s
      #gap
      x = self.gap(x)
      x = self.FC(x)
      x=x.view(-1,10)
      return F.log_softmax(x, dim= -1)



# Summary

In [7]:

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 22, 32, 32]             594
              ReLU-2           [-1, 22, 32, 32]               0
       BatchNorm2d-3           [-1, 22, 32, 32]              44
           Dropout-4           [-1, 22, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           6,336
              ReLU-6           [-1, 32, 32, 32]               0
       BatchNorm2d-7           [-1, 32, 32, 32]              64
           Dropout-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 22, 32, 32]             704
        MaxPool2d-10           [-1, 22, 16, 16]               0
           Conv2d-11           [-1, 32, 16, 16]           6,336
             ReLU-12           [-1, 32, 16, 16]               0
      BatchNorm2d-13           [-1, 32, 16, 16]              64
          Dropout-14           [-1

# Training

In [8]:
from training import *
#Training(3,model)
Training(20,model,device, trainloader, testloader )


EPOCH: 0


Loss=1.8141448497772217 Batch_id=390 Accuracy=25.93: 100%|██████████| 391/391 [00:51<00:00,  7.62it/s]



Test set: Average loss: 2.0513, Accuracy: 2488/10000 (24.88%)

EPOCH: 1


Loss=1.5728487968444824 Batch_id=390 Accuracy=38.57: 100%|██████████| 391/391 [00:48<00:00,  8.09it/s]



Test set: Average loss: 1.7705, Accuracy: 3367/10000 (33.67%)

EPOCH: 2


...


Test set: Average loss: 0.6997, Accuracy: 7616/10000 (76.16%)

EPOCH: 18


Loss=0.7664279341697693 Batch_id=390 Accuracy=75.16: 100%|██████████| 391/391 [00:49<00:00,  7.88it/s]



Test set: Average loss: 0.6799, Accuracy: 7657/10000 (76.57%)

EPOCH: 19


Loss=0.7360885143280029 Batch_id=390 Accuracy=75.50: 100%|██████████| 391/391 [00:48<00:00,  8.04it/s]



Test set: Average loss: 0.6851, Accuracy: 7654/10000 (76.54%)



# 4. Test Accuracy of each class






In [9]:
ClassTestAccuracy(testloader,device,model, classes)

Accuracy of plane : 88 %
Accuracy of   car : 100 %
Accuracy of  bird : 55 %
Accuracy of   cat : 72 %
Accuracy of  deer : 69 %
Accuracy of   dog : 81 %
Accuracy of  frog : 83 %
Accuracy of horse : 78 %
Accuracy of  ship : 85 %
Accuracy of truck : 86 %
